In [1]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool, Tool, BaseTool
from langchain.agents import initialize_agent, AgentType, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from pydantic import BaseModel, Field

In [ ]:
"""
How (STRUCTURED_CHAT_)ZERO_SHOT_REACT_DESCRIPTION agent works:
    next action = agent chooses Function
    while next action is AgentFinish:
        observation = agent uses Parser and runs Function in my computer
        next action = agent chooses Function or AgentFinish based on previous actions and observations
"""
"""
When (STRUCTURED_CHAT_)ZERO_SHOT_REACT_DESCRIPTION agent goes wrong:
    when llm misses format (ex.JSON) so it cannot be parsed
    ex 1) Action:
          ```
          {
            "action": "Sum Calculator",
            "action_input": {
               "a": 355.39,
               "b": 924.87
            }
          }
          ```
    ex 2) Action: Sum Calculator
          Action Input: 355.39, 924.87
"""
"""
OPENAI_FUNCTIONS does not have not much prompt and llm automatically creates function formatting
"""

In [ ]:
def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)

agent = initialize_agent(
    llm=ChatOpenAI(temperature=0.1),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # only work functions that have single input 
    handle_parsing_errors=True, # recover from parsing errors in case llm does not follow the format
    tools=[
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. Use this tool by sending a pair of number separated by a comma.\nExample:1,2",
        ),
    ],
    verbose=True,
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"
agent.invoke(prompt)

In [ ]:
def plus(a, b):
    return a + b

agent = initialize_agent(
    llm=ChatOpenAI(temperature=0.1, model="gpt-4.1-nano"),
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, # work with functions that have multiple input 
    tools=[
        StructuredTool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. This tool takes two arguments, both should be numbers.",
        ),
    ],
    verbose=True,
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"
agent.invoke(prompt)

In [ ]:
class CalculatorToolArgsSchema(BaseModel): # schema
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")

class CalculatorTool(BaseTool): # function call
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers. 
    The first and second arguments should be numbers. 
    Only receives two arguments.
    """
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema # arguments(a,b) should follow CalculatorToolArgsSchema

    def _run(self, a, b): # runs when agent chooses this function
        return a + b

agent = initialize_agent(
    llm=ChatOpenAI(temperature=0.1, model="gpt-4.1-nano"),
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        CalculatorTool()
    ],
    verbose=True,
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"
agent.invoke(prompt)

In [4]:
llm=ChatOpenAI(temperature=0.1, model="gpt-4.1-mini")
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent.invoke("Give me the movies that have the highest votes but the lowest budgets and give me the name of their direcrtors also include their gross revenue.")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: directors, movies
Thought:Thought: There are two tables: directors and movies. The movies table likely contains information about votes, budgets, and gross revenue. The directors table likely contains director names. I should check the schema of both tables to confirm the relevant columns.

Action: sql_db_schema
Action Input: directors, movies
Observation: 
CREATE TABLE directors (
	name TEXT, 
	id INTEGER, 
	gender INTEGER, 
	uid INTEGER, 
	department TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from directors table:
name	id	gender	uid	department
James Cameron	4762	2	2710	Directing
Gore Verbinski	4763	2	1704	Directing
Sam Mendes	4764	2	39	Directing
*/


CREATE TABLE movies (
	id INTEGER, 
	original_title VARCHAR, 
	budget INTEGER, 
	popularity INTEGER, 
	release_date TEXT, 
	revenue INTEGER, 
	title TEXT, 
	vote_average REAL, 
	vote_count INTEGER, 
	overview TEXT, 
	tagline TEXT, 
	uid INTEGER, 
	d

{'input': 'Give me the movies that have the highest votes but the lowest budgets and give me the name of their direcrtors also include their gross revenue.',
 'output': "Here are the top 10 movies with the highest votes but the lowest budgets, along with their directors and gross revenue:\n\n1. Avatar - Votes: 11800, Budget: 237000000, Revenue: 2787965087, Director: James Cameron  \n2. Pirates of the Caribbean: At World's End - Votes: 4500, Budget: 300000000, Revenue: 961000000, Director: Gore Verbinski  \n3. Spectre - Votes: 4466, Budget: 245000000, Revenue: 880674609, Director: Sam Mendes  \n\n(Note: Only 3 movies were shown in the sample data, so these are the top results based on the available data.)"}